# 주제 / 제목 문제

### 참고자료
* Sentence-Transformer: https://huggingface.co/sentence-transformers/bert-base-nli-mean-tokens
* TextRank: yukyunglee Github > https://github.com/yukyunglee/TextRank
* Document Similarity: https://towardsdatascience.com/calculating-document-similarities-using-bert-and-other-models-b2c1a29c9630
* BERT Word Embedding: https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
* BERT Code Visual Guide: https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

In [2]:
import torch
from torch.nn.functional import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer, BertModel, AutoTokenizer, AutoModel

In [3]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [23]:
def sentence_to_bert_wordemb(sentence):
    marked_sentence = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_sentence)
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]
        
    token_embeddings = torch.stack(hidden_states, dim=0)

    token_embeddings.size()

    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)
        
    return token_vecs_sum

    # print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
    
    
# Sentence Format: 'i need calcium and something else for replenish myself'
def sentence_to_bert_sentemb(sentence):
    marked_sentence = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_sentence)
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]

    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = hidden_states[-2][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return sentence_embedding

In [27]:
word_embedding = sentence_to_bert_wordemb('i need calcium and something else to do')
sentence_embddding = sentence_to_bert_sentemb('i need calcium and something else to do')

In [28]:
sentences = test_text.split('.')
sentences = [s.strip()+'.' for s in sentences][:-1]

NameError: name 'test_text' is not defined

### Sentence Embedding

### 다음 글의 제목으로 가장 적절한 것은?

In [31]:
# Answer: 3
test_text = "People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes. Place a small ordinary object in his palm ― a ring, an eraser, anything will do ― and ask him to identify it without moving any part of his hand. He won’t have a clue other than weight and maybe overall size. Then tell him to keep his eyes closed and move his fingers over the object. He’ll most likely identify it at once. By allowing the fingers to move, you’ve added time to the sensory perception of touch. There’s a direct analogy between the fovea at the center of your retina and your fingertips, both of which have high acuity. Your ability to make complex use of touch, such as buttoning your shirt or unlocking your front door in the dark, depends on continuous time-varying patterns of touch sensation."
print(test_text)

one = sentence_to_bert_sentemb('Touch and Movement: Two Major Elements of Humanity')
two = sentence_to_bert_sentemb('The Role of Touch in Forming the Concept of Time')
thr = sentence_to_bert_sentemb('Time Does Matter: A Hidden Essence of Touch')
fou = sentence_to_bert_sentemb('How to Use the Five Senses in a Timely Manner')
fiv = sentence_to_bert_sentemb('The Surprising Function of Touch as a Booster of Knowledge')

answers = [one, two, thr, fou, fiv]
# for idx, answer in enumerate(answers):
#     print(f'({idx}): {answer}')

People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes. Place a small ordinary object in his palm ― a ring, an eraser, anything will do ― and ask him to identify it without moving any part of his hand. He won’t have a clue other than weight and maybe overall size. Then tell him to keep his eyes closed and move his fingers over the object. He’ll most likely identify it at once. By allowing the fingers to move, you’ve added time to the sensory perception of touch. There’s a direct analogy between the fovea at the center of your retina and your fingertips, both of which have high acuity. Your ability to make complex use of touch, such as buttoning your shirt or unlocking your front door in the dark, depends on continuous time-varying patterns of touch sensation.


### 다음 글의 주제로 가장 적절한 것은?

In [182]:
test_text = "Difficulties arise when we do not think of people and machines as collaborative systems, but assign whatever tasks can be automated to the machines and leave the rest to people. This ends up requiring people to behave in machine-like fashion, in ways that differ from human capabilities. We expect people to monitor machines, which means keeping alert for long periods, something we are bad at. We require people to do repeated operations with the extreme precision and accuracy required by machines, again something we are not good at. When we divide up the machine and human components of a task in this way, we fail to take advantage of human strengths and capabilities but instead rely upon areas where we are genetically, biologically unsuited. Yet, when people fail, they are blamed."
print(test_text)

one = sentence_to_bert_sentemb('difficulties of overcoming human weaknesses to avoid failure')
two = sentence_to_bert_sentemb('issues of allocating unfit tasks to humans in automated systems')
thr = sentence_to_bert_sentemb('benefits of allowing machines and humans to work together')
fou = sentence_to_bert_sentemb('reasons why humans continue to pursue machine automation')
fiv = sentence_to_bert_sentemb('influences of human actions on a machine’s performance')

Difficulties arise when we do not think of people and machines as collaborative systems, but assign whatever tasks can be automated to the machines and leave the rest to people. This ends up requiring people to behave in machine-like fashion, in ways that differ from human capabilities. We expect people to monitor machines, which means keeping alert for long periods, something we are bad at. We require people to do repeated operations with the extreme precision and accuracy required by machines, again something we are not good at. When we divide up the machine and human components of a task in this way, we fail to take advantage of human strengths and capabilities but instead rely upon areas where we are genetically, biologically unsuited. Yet, when people fail, they are blamed.


In [197]:
test_text = "Human beings do not enter the world as competent moral agents. Nor does everyone leave the world in that state. But somewhere in between, most people acquire a bit of decency that qualifies them for membership in the community of moral agents. Genes, development, and learning all contribute to the process of becoming a decent human being. The interaction between nature and nurture is, however, highly complex, and developmental biologists are only just beginning to grasp just how complex it is. Without the context provided by cells, organisms, social groups, and culture, DNA is inert. Anyone who says that people are “genetically programmed” to be moral has an oversimplified view of how genes work. Genes and environment interact in ways that make it nonsensical to think that the process of moral development in children, or any other developmental process, can be discussed in terms of nature versus nurture. Developmental biologists now know that it is really both, or nature through nurture. A complete scientific explanation of moral evolution and development in the human species is a very long way off."
print(test_text)

one = sentence_to_bert_sentemb('evolution of human morality from a cultural perspective ')
two = sentence_to_bert_sentemb('difficulties in studying the evolutionary process of genes')
thr = sentence_to_bert_sentemb('increasing necessity of educating children as moral agents')
fou = sentence_to_bert_sentemb('nature versus nurture controversies in developmental biology')
fiv = sentence_to_bert_sentemb('complicated gene-environment interplay in moral development ')

Human beings do not enter the world as competent moral agents. Nor does everyone leave the world in that state. But somewhere in between, most people acquire a bit of decency that qualifies them for membership in the community of moral agents. Genes, development, and learning all contribute to the process of becoming a decent human being. The interaction between nature and nurture is, however, highly complex, and developmental biologists are only just beginning to grasp just how complex it is. Without the context provided by cells, organisms, social groups, and culture, DNA is inert. Anyone who says that people are “genetically programmed” to be moral has an oversimplified view of how genes work. Genes and environment interact in ways that make it nonsensical to think that the process of moral development in children, or any other developmental process, can be discussed in terms of nature versus nurture. Developmental biologists now know that it is really both, or nature through nurtur

In [48]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

# 코사인 유사도 함수 
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def return_max_similarity(summary, answers):
    maxi = 0
    for i, sent_emb in enumerate(answers):
        similarity = cos_sim(summary, sent_emb)
        print(f'answer {i}\'s similarity with summary is {similarity}')
        
        if maxi < similarity:
            maxi = similarity

    return maxi

# return_max(summary, answers)

In [51]:
import re
import itertools
import networkx as nx

from transformers import AutoTokenizer, AutoModel

sent_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-cls-token")
sent_model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-cls-token")    

def sentence_to_bert_emb(sentence):
    marked_sentence = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(add_special_token(marked_sentence))
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]

    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = hidden_states[-2][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return sentence_embedding


def sentences_to_bert_sentemb(sentences):
    encoded_input = sent_tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        model_output = sent_model(**encoded_input)
        sentence_embeddings = model_output[0][:,0] #Take the first token ([CLS]) from each sentence 

    return sentence_embeddings

def make_sentence_graph(sentence, min_sim):
    sentence_graph = nx.Graph()  # initialize an undirected graph
    sentence_graph.add_nodes_from(sentence)

    nodePairs = list(itertools.combinations(sentence, 2))

    # add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        node1 = pair[0]
        node2 = pair[1]

        cos_sim = dot(sentence[pair[0]][1], sentence[pair[1]][1]) / (
            norm(sentence[pair[0]][1]) * norm(sentence[pair[1]][1])
        )

        if cos_sim > min_sim:
            sentence_graph.add_edge(node1, node2, weight=cos_sim)

    return sentence_graph


def extract_sentence(sentence_graph, sentence, top_k):
    calculated_page_rank = nx.pagerank(
        sentence_graph, alpha=0.85, max_iter=100, weight="weight"
    )

    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)

    modified_sentence = sentences[: -len(sentences) + top_k]
    result_sentence = [(sentence[sent][0], sent) for sent in modified_sentence]

    return result_sentence

def sentence_summary(article, min_sim, top_k, cleaned = True):

    sentence_sum_result = []

    original_sentences = [s for s in article.split('.')]

    # sentence마다의 embedding이 포함되어 있다.
    article_embs = [sentence_to_bert_sentemb(s) for s in original_sentences]
    sentence = {}

    for idx, sent in enumerate(article_embs):
        sentence[original_sentences[idx]] = [idx, sent]

    sentence_graph = make_sentence_graph(sentence, min_sim=min_sim)

    extracted_sentence = extract_sentence(sentence_graph, sentence, top_k=top_k)
    sentence_sum_result.append(extracted_sentence)
    
    if cleaned:
        sentence_sum_result = [i[1] for i in sentence_sum_result[0]]
        sentence_sum_result = '.'.join(sentence_sum_result)

    return sentence_sum_result

# need the whold article / not a single sentence
def sentence_summary_sentemb(article, min_sim, top_k, cleaned=True):
    sentence_sum_result = []
    original_sentences = [i.strip() for i in article.split('.')][:-1]
    
    # sentence마다의 embedding이 포함되어 있다.
    article_embs = sentences_to_bert_sentemb(original_sentences)
    sentence = {}

    for idx, sent in enumerate(article_embs):
        sentence[original_sentences[idx]] = [idx, sent]

    sentence_graph = make_sentence_graph(sentence, min_sim=min_sim)

    extracted_sentence = extract_sentence(sentence_graph, sentence, top_k=top_k)
    sentence_sum_result.append(extracted_sentence)
    
    if cleaned:
        sentence_sum_result = [i[1] for i in sentence_sum_result[0]]
        sentence_sum_result = '.'.join(sentence_sum_result)

    return sentence_sum_result

In [52]:
# Answer: 3
test_text = "People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes. Place a small ordinary object in his palm ― a ring, an eraser, anything will do ― and ask him to identify it without moving any part of his hand. He won’t have a clue other than weight and maybe overall size. Then tell him to keep his eyes closed and move his fingers over the object. He’ll most likely identify it at once. By allowing the fingers to move, you’ve added time to the sensory perception of touch. There’s a direct analogy between the fovea at the center of your retina and your fingertips, both of which have high acuity. Your ability to make complex use of touch, such as buttoning your shirt or unlocking your front door in the dark, depends on continuous time-varying patterns of touch sensation."
print(test_text)

one = sentence_to_bert_sentemb('Touch and Movement: Two Major Elements of Humanity')
two = sentence_to_bert_sentemb('The Role of Touch in Forming the Concept of Time')
thr = sentence_to_bert_sentemb('Time Does Matter: A Hidden Essence of Touch')
fou = sentence_to_bert_sentemb('How to Use the Five Senses in a Timely Manner')
fiv = sentence_to_bert_sentemb('The Surprising Function of Touch as a Booster of Knowledge')

answers = [one, two, thr, fou, fiv]

People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes. Place a small ordinary object in his palm ― a ring, an eraser, anything will do ― and ask him to identify it without moving any part of his hand. He won’t have a clue other than weight and maybe overall size. Then tell him to keep his eyes closed and move his fingers over the object. He’ll most likely identify it at once. By allowing the fingers to move, you’ve added time to the sensory perception of touch. There’s a direct analogy between the fovea at the center of your retina and your fingertips, both of which have high acuity. Your ability to make complex use of touch, such as buttoning your shirt or unlocking your front door in the dark, depends on continuous time-varying patterns of touch sensation.


In [58]:
summary = sentence_summary(test_text, min_sim=0.9, top_k=3, cleaned = True)
sentemb_summary = sentence_summary_sentemb(test_text, min_sim=0.9, top_k=3, cleaned = True)

In [59]:
print(summary)
print(sentemb_summary)

People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes
People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial.You can carry out an experiment to see for yourself.Ask a friend to cup his hand, palm face up, and close his eyes


In [55]:
summary = sentence_to_bert_sentemb(summary)

In [56]:
return_max_similarity(summary, answers)

answer 0's similarity with summary is 0.7587635517120361
answer 1's similarity with summary is 0.7747366428375244
answer 2's similarity with summary is 0.7757629156112671
answer 3's similarity with summary is 0.7234643697738647
answer 4's similarity with summary is 0.7858073115348816


0.7858073

# 실제 문제 Test

In [73]:
# Answer: 3
test_text = "People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes. Place a small ordinary object in his palm ― a ring, an eraser, anything will do ― and ask him to identify it without moving any part of his hand. He won’t have a clue other than weight and maybe overall size. Then tell him to keep his eyes closed and move his fingers over the object. He’ll most likely identify it at once. By allowing the fingers to move, you’ve added time to the sensory perception of touch. There’s a direct analogy between the fovea at the center of your retina and your fingertips, both of which have high acuity. Your ability to make complex use of touch, such as buttoning your shirt or unlocking your front door in the dark, depends on continuous time-varying patterns of touch sensation."
print(test_text)

one = sentence_to_bert_sentemb('Touch and Movement: Two Major Elements of Humanity')
two = sentence_to_bert_sentemb('The Role of Touch in Forming the Concept of Time')
thr = sentence_to_bert_sentemb('Time Does Matter: A Hidden Essence of Touch')
fou = sentence_to_bert_sentemb('How to Use the Five Senses in a Timely Manner')
fiv = sentence_to_bert_sentemb('The Surprising Function of Touch as a Booster of Knowledge')

answers = [one, two, thr, fou, fiv]
# for idx, answer in enumerate(answers):
#     print(f'({idx}): {answer}')

People don’t usually think of touch as a temporal phenomenon, but it is every bit as time-based as it is spatial. You can carry out an experiment to see for yourself. Ask a friend to cup his hand, palm face up, and close his eyes. Place a small ordinary object in his palm ― a ring, an eraser, anything will do ― and ask him to identify it without moving any part of his hand. He won’t have a clue other than weight and maybe overall size. Then tell him to keep his eyes closed and move his fingers over the object. He’ll most likely identify it at once. By allowing the fingers to move, you’ve added time to the sensory perception of touch. There’s a direct analogy between the fovea at the center of your retina and your fingertips, both of which have high acuity. Your ability to make complex use of touch, such as buttoning your shirt or unlocking your front door in the dark, depends on continuous time-varying patterns of touch sensation.


In [75]:
# 2021 영어 영역 짝수형 23
# 정답 2

test_text = "Difficulties arise when we do not think of people and machines as collaborative systems, but assign whatever tasks can be automated to the machines and leave the rest to people. This ends up requiring people to behave in machine-like fashion, in ways that differ from human capabilities. We expect people to monitor machines, which means keeping alert for long periods, something we are bad at. We require people to do repeated operations with the extreme precision and accuracy required by machines, again something we are not good at. When we divide up the machine and human components of a task in this way, we fail to take advantage of human strengths and capabilities but instead rely upon areas where we are genetically, biologically unsuited. Yet, when people fail, they are blamed."
print(test_text)

one = sentence_to_bert_sentemb('difficulties of overcoming human weaknesses to avoid failure')
two = sentence_to_bert_sentemb('issues of allocating unfit tasks to humans in automated systems')
thr = sentence_to_bert_sentemb('benefits of allowing machines and humans to work together')
fou = sentence_to_bert_sentemb('reasons why humans continue to pursue machine automation')
fiv = sentence_to_bert_sentemb('influences of human actions on a machine’s performance')

Difficulties arise when we do not think of people and machines as collaborative systems, but assign whatever tasks can be automated to the machines and leave the rest to people. This ends up requiring people to behave in machine-like fashion, in ways that differ from human capabilities. We expect people to monitor machines, which means keeping alert for long periods, something we are bad at. We require people to do repeated operations with the extreme precision and accuracy required by machines, again something we are not good at. When we divide up the machine and human components of a task in this way, we fail to take advantage of human strengths and capabilities but instead rely upon areas where we are genetically, biologically unsuited. Yet, when people fail, they are blamed.


In [71]:
# 2020 영어 영역 홀수형 23
# 정답 5

test_text = "Human beings do not enter the world as competent moral agents. Nor does everyone leave the world in that state. But somewhere in between, most people acquire a bit of decency that qualifies them for membership in the community of moral agents. Genes, development, and learning all contribute to the process of becoming a decent human being. The interaction between nature and nurture is, however, highly complex, and developmental biologists are only just beginning to grasp just how complex it is. Without the context provided by cells, organisms, social groups, and culture, DNA is inert. Anyone who says that people are “genetically programmed” to be moral has an oversimplified view of how genes work. Genes and environment interact in ways that make it nonsensical to think that the process of moral development in children, or any other developmental process, can be discussed in terms of nature versus nurture. Developmental biologists now know that it is really both, or nature through nurture. A complete scientific explanation of moral evolution and development in the human species is a very long way off."
print(test_text)

one = sentence_to_bert_sentemb('evolution of human morality from a cultural perspective ')
two = sentence_to_bert_sentemb('difficulties in studying the evolutionary process of genes')
thr = sentence_to_bert_sentemb('increasing necessity of educating children as moral agents')
fou = sentence_to_bert_sentemb('nature versus nurture controversies in developmental biology')
fiv = sentence_to_bert_sentemb('complicated gene-environment interplay in moral development ')

Human beings do not enter the world as competent moral agents. Nor does everyone leave the world in that state. But somewhere in between, most people acquire a bit of decency that qualifies them for membership in the community of moral agents. Genes, development, and learning all contribute to the process of becoming a decent human being. The interaction between nature and nurture is, however, highly complex, and developmental biologists are only just beginning to grasp just how complex it is. Without the context provided by cells, organisms, social groups, and culture, DNA is inert. Anyone who says that people are “genetically programmed” to be moral has an oversimplified view of how genes work. Genes and environment interact in ways that make it nonsensical to think that the process of moral development in children, or any other developmental process, can be discussed in terms of nature versus nurture. Developmental biologists now know that it is really both, or nature through nurtur

In [76]:
summary = sentence_summary(test_text, min_sim=0.9, top_k=3, cleaned = True)
sentemb_summary = sentence_summary_sentemb(test_text, min_sim=0.9, top_k=3, cleaned = True)

print(summary)
print(sentemb_summary)

summary = sentence_to_bert_sentemb(summary)
return_max_similarity(summary, answers)

Difficulties arise when we do not think of people and machines as collaborative systems, but assign whatever tasks can be automated to the machines and leave the rest to people. When we divide up the machine and human components of a task in this way, we fail to take advantage of human strengths and capabilities but instead rely upon areas where we are genetically, biologically unsuited. This ends up requiring people to behave in machine-like fashion, in ways that differ from human capabilities
Difficulties arise when we do not think of people and machines as collaborative systems, but assign whatever tasks can be automated to the machines and leave the rest to people.This ends up requiring people to behave in machine-like fashion, in ways that differ from human capabilities.We expect people to monitor machines, which means keeping alert for long periods, something we are bad at
answer 0's similarity with summary is 0.7676581740379333
answer 1's similarity with summary is 0.74345874786

0.7676582